<a href="https://colab.research.google.com/github/MJSahebnasi/temporal-network-platform/blob/main/thesis_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import networkx as nx

from typing import List, Type

import copy

# **Model**

## Events

In [30]:
class Event:
  def __init__(self, t) -> None:
    self.time = t
    
  def __repr__(self):
    return "<time: "+str(self.time)+">"

class NodeEvent(Event):
  def __init__(self, t, node) -> None:
    super().__init__(t)
    self.node = node

  def __repr__(self):
    return super().__repr__()[:-1] + " - node: "+str(self.node) + ">"

class NodeEntranceEvent(NodeEvent):
  def __init__(self, t, node, neighbors) -> None:
    super().__init__(t, node)
    self.neighbors = neighbors

  def __repr__(self):
    return super().__repr__()[:-1] + " - entrance" + ">"

  # need id (or other info) for edges?

class NodeNaturalDeathEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)

class NodeAttackEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)

class NodeOnEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)

class NodeOffEvent(NodeEvent):
  def __init__(self, t, node) -> None:
    super().__init__(t, node)


## Simulation

In [29]:
class Simulation:   

  ########################## init ##########################

  def create(self, G):
    self.orig_G = G
    self.sim_G = copy.deepcopy(G)

    self.has_entrance_process = False
    self.has_onoff_process = False
    self.has_node_lifespan = False
    self.has_attack_process = False

    self.events = []
    # self.history = []
    # self.snapshots = []

    return self

  def with_entrance_process(self, node_entrance_times):
    ### TODO: parameter validation ###
    self.has_entrance_process = True
    self.node_entrance_times = node_entrance_times
    return self

  def with_node_onoff_process(self, node_on_times, node_off_times):
    ### TODO: parameter validation ###
    self.has_onoff_process = True
    self.node_on_times = node_on_times
    self.node_off_times = node_off_times
    return self

  def with_node_lifespans(self, node_death_times):
    ### TODO: parameter validation ###
    self.has_node_lifespan = True
    self.node_death_times = node_death_times
    return self

  def with_attack_process(self, node_attack_times):
    ### TODO: parameter validation ###
    self.has_attack_process = True
    self.node_attack_times = node_attack_times
    return self


  ########################## prepair events ##########################


  def create_entrance_events(self):
    n = len(self.orig_G.nodes())
    nodes = list(self.orig_G.nodes())
    self.sim_G = nx.Graph()
    
    ### later, move to init part
    if len(self.node_entrance_times) != n:
      raise ValueError('!!! len(node_entrance_times) & len(G.nodes) not the same !!!')
    ###

    # dunno how nx handles it, guess set might be faster
    sim_G_nodes = set()
    for i in range(n):
      node = nodes[i]
      sim_G_nodes.add(node)
      neighbors = set.intersection(set(nbr for nbr in self.orig_G.neighbors(i)), sim_G_nodes)
      e = NodeEntranceEvent(self.node_entrance_times[i], node, neighbors)
      self.events.append(e)

  def create_onoff_events(self):
    pass ### TODO: create + add to self.events

  def create_death_events(self):
    pass ### TODO: create + add to self.events

  def create_attack_events(self):
    ##########
    # r = self.attack_time_generator(**self.attack_time_generator_args_dict)
    # print('attack:', r)
    ##########
    pass ### TODO: create + add to self.events

  def prepair_events(self):
    if self.has_entrance_process:
      self.create_entrance_events()

    if self.has_onoff_process:
      self.create_onoff_events()

    if self.has_node_lifespan:
      self.create_death_events()

    if self.has_attack_process:
      self.create_attack_events()

    ### TODO: heapify events


  ########################## run ##########################


  def run(self):
    self.prepair_events()
    sim_time = 0
    
    ###
    print(self.events)
    ###
    
    # while len(self.events) > 0:
      # e = min of "events heap"
      # e.execute()
      # sim_time = e.time

      # handle history/snapshot
      # pass


